In [15]:
import os
import glob
import json
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import requests


In [16]:
workdir = os.path.dirname(os.getcwd())
input_dir = Path(f"{workdir}/8_drug_protein_moa/data/")
module_path = Path("/home/bbc8731/HSV/3_module_expansion/data/categories_methods")

In [18]:
module_files = sorted(module_path.rglob("*/drug_repurposing/trustrank/uniprot_ppi/validation/approved_drugs/uniprot_ppi_drug_validation.json"))
for p in module_files:
    base_path = p.parents[0]
    print(base_path)
    with open(p, "r") as f:
        data = json.load(f)
        
    predicted_drugs = data["test_drugs"]
    predicted_drugs = pd.DataFrame(predicted_drugs, columns=["drug_id", "score"])
    predicted_drugs["score"] = predicted_drugs["score"].astype(float)
    predicted_drugs["neg_log_score"] = -np.log10(predicted_drugs["score"])
    predicted_drugs = (predicted_drugs.sort_values("score", ascending=False).reset_index(drop=True))
    predicted_drugs["trustrank"] = predicted_drugs["neg_log_score"].rank(method="min",ascending=True)
    predicted_drugs["trustrank"] = predicted_drugs["trustrank"].astype(int)

    output_file = base_path / f"candidate_drugs_scored.csv"
    predicted_drugs.to_csv(output_file, index=False)


/home/bbc8731/HSV/3_module_expansion/data/categories_methods/BP_Assembly_and_Packaging/drug_repurposing/trustrank/uniprot_ppi/validation/approved_drugs
/home/bbc8731/HSV/3_module_expansion/data/categories_methods/BP_Egress_and_Envelopment/drug_repurposing/trustrank/uniprot_ppi/validation/approved_drugs
/home/bbc8731/HSV/3_module_expansion/data/categories_methods/BP_Entry_and_Uncoating/drug_repurposing/trustrank/uniprot_ppi/validation/approved_drugs
/home/bbc8731/HSV/3_module_expansion/data/categories_methods/BP_Immune_Evasion/drug_repurposing/trustrank/uniprot_ppi/validation/approved_drugs
/home/bbc8731/HSV/3_module_expansion/data/categories_methods/BP_Replication_and_Transcription/drug_repurposing/trustrank/uniprot_ppi/validation/approved_drugs
/home/bbc8731/HSV/3_module_expansion/data/categories_methods/BP_Uncharacterized/drug_repurposing/trustrank/uniprot_ppi/validation/approved_drugs
/home/bbc8731/HSV/3_module_expansion/data/categories_methods/CC_Host_Cytoplasm/drug_repurposing/tru

In [19]:
predicted_drugs

,drug_id,score,neg_log_score,trustrank
0,drugbank.DB01268,0.000102,3.989723,1
1,drugbank.DB06595,0.000089,4.051945,2
2,drugbank.DB12010,0.000072,4.140828,3
3,drugbank.DB11829,0.000059,4.231666,4
4,drugbank.DB09079,0.000057,4.244399,5
...,...,...,...,...
57,drugbank.DB08868,0.000002,5.628978,58
58,drugbank.DB00175,0.000002,5.673410,59
59,drugbank.DB09091,0.000002,5.680761,60
60,drugbank.DB03904,0.000002,5.715201,61
